In [1]:
import pandas as pd

import os

import numpy as np

import networkx as nx

from IPython.display import clear_output

# Nodes

In [2]:
df_comments = pd.read_csv('data/get_comment_threads_output/comments_thread_sentiment_reduced_5.csv')

In [3]:
user_name_set= set()
nan_number=0
for comment in df_comments.values:
    if comment[2]!= None:
        user_name_set.add(comment[2])
    else:
        nan_number+=1

In [4]:
first_graph = nx.Graph()
tot_nodes= list(user_name_set)
#tolgo il nan
tot_nodes = tot_nodes[1:]
first_graph.add_nodes_from(tot_nodes)

In [5]:
gephi_graph = nx.Graph()
tot_nodes= list(user_name_set)
#tolgo il nan
tot_nodes = tot_nodes[1:]
gephi_graph.add_nodes_from(tot_nodes)

In [6]:
df_youtuber_videos = pd.read_csv('data/get_youtuber_videos_output/youtuber_videos.csv')
df_youtubers = pd.read_csv('data/get_youtubers_output/youtubers.csv')
df_comments = pd.read_csv('data/get_comment_threads_output/comments_thread_sentiment_reduced_5.csv')
df_videos = pd.read_csv('data/get_videos_details/videos_details_computed.csv')
youtuber_video_join=df_youtubers.set_index('channel_id').join(df_youtuber_videos.set_index('channel_id'), on='channel_id',lsuffix='_yt', rsuffix='_ytvd')
youtuber_video_join = youtuber_video_join[youtuber_video_join.columns[[0,2]]]
youtuber_video_join = youtuber_video_join.set_index('video_id').join(df_videos.set_index('video_id'),on='video_id',lsuffix='_yt', rsuffix='_ytvd')

In [7]:
topics_set =set()
brands_set = set()
youtubers_set = set()

In [8]:
#per set
for idx, n in enumerate(first_graph.nodes):
    print('{}/{}'.format(idx,len(first_graph.nodes)))
    clear_output(wait=True)
    topics = []
    brands= []
    youtubers= []
    user_name = n
    video_idx = df_comments.set_index('author_display_name').loc[[user_name],['video_id']]['video_id'][0]
    row = youtuber_video_join.loc[[video_idx],:]
    [topics.append(item.strip()[1:-1]) for item in row['topics'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['topics'][0]) is str else []
    [brands.append(item.strip()[1:-1]) for item in row['brands'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['brands'][0]) is str else []
    [youtubers.append(item.strip()[1:-1]) for item in row['youtubers'][0][1:-1].split(',') if item.strip()[1:-1]!= ""] if type(row['youtubers'][0]) is str else []
    if row['display_name'][0].strip()!= "":        
        youtubers.append(row['display_name'][0])
    [topics_set.add(item) for item in topics]
    [brands_set.add(item) for item in brands]
    [youtubers_set.add(item) for item in youtubers]

10575/10576


In [9]:
topics_complete = topics_set.union(brands_set).union(youtubers_set)
len(topics_complete)

238

In [10]:
for n in gephi_graph.nodes:
    for t in topics_complete:
        gephi_graph.nodes[n][t] = 0
        first_graph.nodes[n][t] = 0

In [11]:
for idx, n in enumerate(first_graph.nodes):
    print('{}/{}'.format(idx,len(first_graph.nodes)))
    clear_output(wait=True)
    topics = []
    brands= []
    youtubers= []
    user_name = n
    video_idx = df_comments.set_index('author_display_name').loc[[user_name],['video_id']]['video_id'][0]
    row = youtuber_video_join.loc[[video_idx],:]
    if type(row['topics'][0]) is str:
        for item in row['topics'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1 
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1 
    if type(row['brands'][0]) is str:
        for item in row['brands'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1 
    if type(row['youtubers'][0]) is str:
        for item in row['youtubers'][0][1:-1].split(','):
            if item.strip()[1:-1]!= "":
                first_graph.nodes[n][item.strip()[1:-1]] = 1 
                gephi_graph.nodes[n][item.strip()[1:-1]] = 1 
    if row['display_name'][0].strip()!= "":  
        first_graph.nodes[n][row['display_name'][0]] = 1
        gephi_graph.nodes[n][row['display_name'][0]] = 1 

10575/10576


# Edges

In [12]:
for i in range(13):
    for j in range(6,10):
        edges = list(np.load('data/get_edges/percentile/percentili_edges_{}_{}.npy'.format(i,j), allow_pickle = True))
        for idx, e in enumerate(edges):
            print('File percentili_edges_{}_{}.npy'.format(i,j))
            print('{}/{}'.format(idx,len(edges)))
            first_graph.add_edge(e[0][0],e[0][1],weight=e[0][2],youtubers = [item for item in e[1]['youtubers']], topics = e[1]['topics'], brands = e[1]['brands'], sentiment = e[1]['sentiment'])
            gephi_graph.add_edge(e[0][0],e[0][1],weight=e[0][2])
            clear_output(wait=True)

File percentili_edges_12_9.npy
86909/86910


In [13]:
gephi_graph.remove_edges_from(nx.selfloop_edges(gephi_graph))
first_graph.remove_edges_from(nx.selfloop_edges(first_graph))

# Salvo

In [14]:
nx.write_gpickle(first_graph,'data/graphs/complete_graph.gpickle')
nx.write_gpickle(gephi_graph,'data/graphs/complete_gephi_graph.gpickle')

In [15]:
nx.write_gexf(gephi_graph,'data/graphs/complete_gephi_graph.gexf')